### `Conv2D` code

In [153]:
import numpy as np

In [154]:
class Conv2D:
    ''' Computes convolution given the input parameters '''
    
    ''' * The class implementation will be along the lines of torch.nn.Conv2D in order to 
          enable comparison of this NumPy only implementation and seamless testing
        * Can expect extensive refactoring of the existing code in the days to come
        * As part of refactoring, some code will be de-modularized
        * Old code will be retained at the end of the notebook for reference
    '''
    '''
        TODO:
        * Add various 'same' `padding` mode
        * Implementing other features and caveats offered by nn.torch.Conv2D 
          (e.g., uniform sampling of kernel weights, bias etc.)
        * Optimizing code
    '''
    
    def __init__(
        self, 
        in_channels, 
        out_channels, 
        kernel_size, 
        padding = (0, 0), 
        stride = (1, 1), 
        dilation = (1, 1), 
        groups = 1, 
        bias = True, 
        padding_mode = 'zeros', 
        device = None, 
        dtype = None, 
        verbose = False, 
        # debug = False
        ):
        super(Conv2D, self).__init__()
        
        ''' mandatory parameters '''
        self.in_channels = in_channels
        self.out_channels = out_channels
        if isinstance(kernel_size, tuple):
            self.kernel_size = kernel_size
        elif isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            raise Exception('invalid input parameters: kernel_size should either be an int or a tuple')
        
        ''' optional parameters '''
        if isinstance(padding, str):
            if padding == 'valid':
                self.padding = (0, 0)
            elif padding == 'same':
                raise Exception("invalid input parameters: padding = 'same' not yet supported")
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, tuple):
            if padding[0] >= 0 and padding[1] >= 0:
                self.padding = padding
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, int):
            if padding >= 0:
                self.padding = (padding, padding)
            else:
                raise Exception('invalid input parameters: padding is not valid')
        else:
            raise Exception('invalid input parametersL padding should be either an int or a tuple')
        if isinstance(stride, tuple):
            if stride[0] >= 1 and stride[1] >= 1:
                self.stride = stride
            else:
                raise Exception('invalid input parameters: stride is not valid')
        elif isinstance(stride, int):
            if stride >= 1:
                self.stride = (stride, stride)
            else:
                raise Exception('invalid input parameters: stride is not valid')
        else:
            raise Exception('invalid input parameters: stride should be either an int or a tuple')
        if isinstance(dilation, tuple):
            if dilation[0] >= 1 and dilation[1] >= 1:
                self.dilation = dilation
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        elif isinstance(dilation, int):
            if dilation >= 1:
                self.dilation = (dilation, dilation)
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        else:
            raise Exception('invalid input parameters: dilation should be either an int or a tuple')
        self.groups = groups
        
        ''' optional parameters (dummy, yet to be implemented)'''
        self.bias = bias
        self.padding_mode = padding_mode
        self.device = device
        self.dtype = dtype
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        # self.debug = debug
        # self.debugprint = print if self.debug else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('in_channels: {}, out_channels: {}, kernel_size: {}'.format(self.in_channels, self.out_channels, self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        self.verboseprint('groups: {}, bias: {}, padding_mode: {}, device: {}, dtype: {}'.format(self.groups, self.bias, self.padding_mode, self.device, self.dtype))
        self.verboseprint('\n')
    
    def forward(self, _input, kernels):
        ''' forward pass to perform convolution '''
        
        ''' do error checking '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        if _input_h + 2 * self.padding[0] < self.dilation[0] * (self.kernel_size[0] - 1) + 1: # check if (dilated) ker_h is valid
            raise Exception('invalid input parameters: kernel height is larger than input height')
        if _input_w + 2 * self.padding[1] < self.dilation[1] * (self.kernel_size[1] - 1) + 1: # check if (dilated) ker_w is valid
            raise Exception('invalid input parameters: kernel width is larger than input width')
        if ((_input_h + 2 * self.padding[0] - (self.dilation[0] * (self.kernel_size[0] - 1) + 1)) / self.stride[0]) + 1 < 0: # check if out_h is valid
            raise Exception('invalid input parameters: output height is negative')
        if ((_input_w + 2 * self.padding[1] - (self.dilation[1] * (self.kernel_size[1] - 1) + 1)) / self.stride[1]) + 1 < 0: # check if out_w is valid
            raise Exception('invalid input parameters: output width is negative')
        if  self.in_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parameters: input channels is not divisible by groups')
        if self.out_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parametes: output channels is not divisible by groups')
         
        ''' add zero padding based on the input parameters '''
        if self.padding != (0, 0):
            _input = np.array([[np.pad(channel, ((self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), 'constant', constant_values = 0) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        # if self.debug:
            # for b in range(_input.shape[0]):
                # _input[b] = (b + 1) * np.ones_like(_input[b]) # define an image of all ones (twos etc.) based on the input parameters
        
        ''' use the provided kernels or create random kernels based on the input kernel parameters '''
        if kernels is not None:
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        else:
            kernels = []
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
            for k in range(self.out_channels):
                kernel = np.random.rand(int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]) # define a random kernel based on the kernel parameters
                # if self.debug:
                    # kernel = k * np.ones_like(kernel)
                kernels.append(kernel)
                self.verboseprint('kernel {}'.format(k))
                self.verboseprint(kernel)
            self.verboseprint('\n')
        
        ''' dilate a kernel '''
        dil_ker_h = self.dilation[0] * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation[1] * (self.kernel_size[1] - 1) + 1
        dil_kernels = []
        for kernel in kernels:
            dil_kernel = []
            for channel in kernel:
                dil_channel = np.zeros((dil_ker_h, dil_ker_w))
                for row in range(len(channel)):
                    for col in range(len(channel[0])):
                        dil_channel[self.dilation[0] * row][self.dilation[1] * col] = channel[row][col] # check if the indices 0 and 1 need to be swapped
                dil_kernel.append(dil_channel.tolist())
            dil_kernels.append(dil_kernel)
        kernels, self.kernel_size = dil_kernels, (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('kernels: {}, dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.dilation, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        for k in range(self.out_channels):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernels[k])
        self.verboseprint('\n')
        
        ''' compute output volume from the input and kernel parameters '''
        _input_n, _, _input_h, _input_w = _input.shape
        out_n = int(_input_n)
        out_c = int(self.out_channels)
        out_h = int((_input_h - self.kernel_size[0]) / self.stride[0]) + 1
        out_w = int((_input_w - self.kernel_size[1]) / self.stride[1]) + 1
        output = np.zeros([out_n, out_c, out_h, out_w])
        
        ''' parse through every element of the output and compute the convolution value for that element '''
        for b in range(out_n):
            for k in range(out_c):
                for h in range(out_h):
                    for w in range(out_w):
                        # convolve kernel over the input slices
                        self.verboseprint('kernel indices, image indices')
                        self.verboseprint('[n, c, h, w]', '[n, c, h, w]')
                        convol_sum = 0
                        ker_c = int(self.in_channels / self.groups)
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        for c_ker in range(ker_c):
                            for h_ker in range(ker_h):
                                for w_ker in range(ker_w):
                                    # if self.groups == 1:
                                        # self.verboseprint([k, c_ker, h_ker, w_ker], [b, c_ker, h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                        # convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] # works only when `groups` == 1
                                    # elif self.groups == self.in_channels:
                                        # self.verboseprint([k, c_ker, h_ker, w_ker], [b, k, h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                        # convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][k][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] # works only when `groups` == in_channels
                                    # else:
                                        # self.verboseprint([k, c_ker, h_ker, w_ker], [b, c_ker if k < self.groups else c_ker + ker_c, h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                        # convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker if k < self.groups else c_ker + ker_c][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] # works in some cases when 1 < `groups` < in_channels
                                    self.verboseprint([k, c_ker, h_ker, w_ker], [b, c_ker + (ker_c * int(np.floor(k / (self.out_channels / self.groups)))), h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                    convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker + (ker_c * int(np.floor(k / (self.out_channels / self.groups))))][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] # works for all 1 <= `groups` <= in_channels
                        self.verboseprint('\n')
                        output[b, k, h, w] += convol_sum
        self.verboseprint('*** Conv2DPytTest output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random kernel, random input)

In [155]:
in_channels = 6 # input channels
out_channels = 4 # output channels
kernel_size = (2, 2) # kernel size

padding = (1, 3) # padding (optional)
stride = (2, 1) # stride (optional)
dilation = (2, 3) # dilation factor (optional)
groups = 2 # groups (optional)

in_batches = 2 # input batches
in_h = 4 # input height
in_w = 4 # input weight

_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
kernels = []
for k in range(out_channels):
    kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
    kernels.append(kernel)

In [156]:
# get Conv2D output with the random inputs

conv2d = Conv2D(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups) # call an instance of the class with the input parameters 
_output = conv2d.forward(_input, kernels) # perform convolution
print("*** Conv2D output ***")
print(_output)

*** Conv2DPytTestGroups output ***
[[[[1.29322222 0.59732106 0.48430893 1.84416206 0.24456822 0.26208592
    0.74219291]
   [1.5728169  1.73119872 1.99582864 3.48284803 1.59447953 1.30855948
    1.28994345]]

  [[1.33145361 1.05347664 0.36701413 2.25878962 0.8785711  0.3185761
    0.96079477]
   [1.35738283 1.55314074 1.58469059 2.52600983 1.04354317 1.53736274
    1.2691641 ]]

  [[1.07841053 1.12654574 0.24065743 1.55465241 1.04126951 0.32803478
    0.49324433]
   [1.41429069 1.71767543 1.56997786 4.11684004 2.24767687 1.50559267
    2.22214463]]

  [[1.63338405 1.48193695 0.93856912 2.1033145  1.14461283 0.69425896
    1.25996082]
   [2.2519336  2.59685906 1.7481735  4.44865206 1.32466051 0.77799261
    1.7717178 ]]]


 [[[0.24994835 0.99618313 1.08422208 0.41399767 0.82080665 0.77060424
    0.08335595]
   [1.62585005 1.76325336 1.89752126 2.75501933 1.55708554 2.19540888
    1.02395867]]

  [[0.41500445 0.40973895 1.16283891 0.68959752 0.80300663 1.401363
    0.19071419]
   [1.5394

In [157]:
# get PyTorch output with the same random inputs as above

import torch

x = torch.DoubleTensor(_input)
weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation, groups = groups)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[1.2932, 0.5973, 0.4843, 1.8442, 0.2446, 0.2621, 0.7422],
          [1.5728, 1.7312, 1.9958, 3.4828, 1.5945, 1.3086, 1.2899]],

         [[1.3315, 1.0535, 0.3670, 2.2588, 0.8786, 0.3186, 0.9608],
          [1.3574, 1.5531, 1.5847, 2.5260, 1.0435, 1.5374, 1.2692]],

         [[1.0784, 1.1265, 0.2407, 1.5547, 1.0413, 0.3280, 0.4932],
          [1.4143, 1.7177, 1.5700, 4.1168, 2.2477, 1.5056, 2.2221]],

         [[1.6334, 1.4819, 0.9386, 2.1033, 1.1446, 0.6943, 1.2600],
          [2.2519, 2.5969, 1.7482, 4.4487, 1.3247, 0.7780, 1.7717]]],


        [[[0.2499, 0.9962, 1.0842, 0.4140, 0.8208, 0.7706, 0.0834],
          [1.6259, 1.7633, 1.8975, 2.7550, 1.5571, 2.1954, 1.0240]],

         [[0.4150, 0.4097, 1.1628, 0.6896, 0.8030, 1.4014, 0.1907],
          [1.5395, 1.5873, 1.3798, 2.8508, 1.7745, 1.1612, 1.5501]],

         [[0.8876, 0.3962, 0.6684, 2.0243, 0.3906, 0.5973, 1.0404],
          [1.4767, 0.6745, 0.7747, 3.7180, 0.8273, 1.0943, 2.5221]],

         

In [158]:
# compare outputs of conv-Numpy and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests (random kernel, random input)

In [159]:
def valid_params(num_tests):
    # generates `num_samples` number of valid input and kernel parameters 
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        out_channels = np.random.randint(20) + 1 # output channels
        
        kernel_h = np.random.randint(20) + 1
        kernel_w = np.random.randint(20) + 1
        kernel_size = (kernel_h, kernel_w) # kernel size
        
        padding_h = np.random.randint(10) + 1
        padding_w = np.random.randint(10) + 1
        padding = (padding_h, padding_w) # padding (optional)
        stride_h = np.random.randint(5) + 1
        stride_w = np.random.randint(5) + 1
        stride = (stride_h, stride_w) # stride (optional)
        dilation_h = np.random.randint(10) + 1
        dilation_w = np.random.randint(10) + 1
        dilation = (dilation_h, dilation_w) # dilation factor (optional)
        groups = np.random.randint(in_channels) + 1 # groups (optional)
        
        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
    
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag, in_group_flag, out_group_flag = True, True, True, True, True, True
        
        if in_h + 2 * padding_h < dilation_h * (kernel_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding_w < dilation_w * (kernel_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding_h - (dilation_h * (kernel_h - 1) + 1)) / stride_h) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding_w - (dilation_w * (kernel_w - 1) + 1)) / stride_w) + 1 < 0: # check if out_w is valid
            out_w_flag = False
        if  in_channels % groups != 0: # check if groups is valid
            in_group_flag = False
        if out_channels % groups != 0: # check if groups is valid
            out_group_flag = False
        
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag and in_group_flag and out_group_flag:
            params_list.append({'in_channels': in_channels, 'out_channels': out_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'groups': groups, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w})
            sample_count += 1
    return params_list

In [160]:
# for loop sweeping different input parameters and testing the outputs of Conv2DPytTestGroups and PyTorch
from tqdm import tqdm

num_tests = 200 # only test parameter to be varied
num_passed = 0
params_list = valid_params(num_tests)
print('Number of tests: {}\n\n'.format(len(params_list)))

for i, params in enumerate(tqdm(params_list)):
    print('Test: {}\nParams: {}'.format(i, params))
    # debug = False # DO NOT CHANGE THIS while using Conv2DPytTest - TODO: modify this flag to `test`

    in_channels = params['in_channels'] # input channels
    out_channels = params['out_channels'] # output channels
    
    kernel_size = params['kernel_size'] # kernel size

    padding = params['padding'] # padding (optional)
    stride = params['stride'] # stride (optional)
    dilation = params['dilation'] # dilation factor (optional)
    groups = params['groups'] # groups (optional)
    
    in_batches = params['in_batches'] # input batches
    in_h = params['in_h'] # input height
    in_w = params['in_w'] # input weight
    
    _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
    kernels = []
    for k in range(out_channels):
        kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
        kernels.append(kernel)
    
    try:
        # get Conv2DPytTest output with the random inputs
        conv2dpyttestgroups = Conv2DPytTestGroups(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups) # call an instance of the class with the input parameters 
        _output = conv2dpyttestgroups.forward(_input, kernels) # perform convolution

        # get PyTorch output with the same random inputs as above
        x = torch.DoubleTensor(_input)
        weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
        output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation, groups = groups)
        
    except Exception as e:
        print(e)
        print('Result: False\n\n') # treating exception as a failed test
        continue
    
    # compare outputs of conv-Numpy and PyTorch
    result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
    print('Result: {}\n\n'.format(result))
    if result:
        num_passed += 1

print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))
    

Number of tests: 200




  0%|                                                   | 0/200 [00:00<?, ?it/s]

Test: 0
Params: {'in_channels': 1, 'out_channels': 16, 'kernel_size': (2, 5), 'padding': (1, 10), 'stride': (1, 1), 'dilation': (10, 7), 'groups': 1, 'in_batches': 4, 'in_h': 32, 'in_w': 14}


  0%|▏                                          | 1/200 [00:15<50:40, 15.28s/it]

Result: True


Test: 1
Params: {'in_channels': 3, 'out_channels': 11, 'kernel_size': (2, 2), 'padding': (1, 10), 'stride': (1, 2), 'dilation': (5, 3), 'groups': 1, 'in_batches': 4, 'in_h': 21, 'in_w': 19}


...

Result: True


Test: 197
Params: {'in_channels': 8, 'out_channels': 4, 'kernel_size': (3, 1), 'padding': (8, 9), 'stride': (4, 4), 'dilation': (3, 5), 'groups': 4, 'in_batches': 5, 'in_h': 28, 'in_w': 28}


 99%|████████████████████████████████████████▌| 198/200 [29:30<00:06,  3.13s/it]

Result: True


Test: 198
Params: {'in_channels': 8, 'out_channels': 7, 'kernel_size': (17, 1), 'padding': (5, 7), 'stride': (5, 5), 'dilation': (1, 6), 'groups': 1, 'in_batches': 1, 'in_h': 21, 'in_w': 6}
Result: True


Test: 199
Params: {'in_channels': 6, 'out_channels': 12, 'kernel_size': (4, 6), 'padding': (10, 10), 'stride': (1, 4), 'dilation': (6, 2), 'groups': 1, 'in_batches': 1, 'in_h': 7, 'in_w': 21}


100%|█████████████████████████████████████████| 200/200 [29:36<00:00,  8.88s/it]

Result: True


200 out of 200 (100.0%) tests passed


### Modular code for methods in `forward()`

### Modular code for `forward()`

### Old `Conv2D` code  for reference